# Apache Scratch Pad


This notebook will aim to query the Augur DB to access the neccessary information to be able to get the following issue metrics dirived from the GitHub Community Metrics working document https://docs.google.com/document/d/1Yocr6fk0J8EsVZnJwoIl3kRQaI94tI-XHe7VSMFT0yM/edit?usp=sharing

Any necessary computations from the data to get the metric value will be done as the queries are determined

In [16]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import json
import os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)
import warnings
warnings.filterwarnings('ignore')

with open("../../comm_cage.json") as config_file:
    config = json.load(config_file)

In [17]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [46]:
#add your repo name(s) here of the repo(s) you want to query if known (and in the database)
repo_name_set = ['augur']
repo_set = []
repo_ids = []

repolist = pd.DataFrame() 

for repo_name in repo_name_set:
    repo_query = salc.sql.text(f"""
                 SET SCHEMA 'augur_data';
                 SELECT 
                    b.repo_id
                FROM
                    repo_groups a,
                    repo b
                WHERE
                    a.repo_group_id = b.repo_group_id AND
                    b.repo_name = \'{repo_name}\'
        """)
    print(repo_query)

    repolist = pd.read_sql(repo_query, con=engine);
    d, a = {}, []
    for repo in repolist:
        # rowproxy.items() returns an array like [(key0, value0), (key1, value1)]
        for column, value in repolist.items():
            # build up the dictionary
            d = {**d, **{column: value}}
        #$repo_set.append(d)
        repo_ids = repolist['repo_id']

    
        
    #repo_id =t.mappings().all()[0].get('repo_id')
    repo_set.append(repolist['repo_id'][0])

# repo_set = repolist['repo_id'][0]
print(repo_set)
print(repolist['repo_id'][0])



                 SET SCHEMA 'augur_data';
                 SELECT 
                    b.repo_id
                FROM
                    repo_groups a,
                    repo b
                WHERE
                    a.repo_group_id = b.repo_group_id AND
                    b.repo_name = 'augur'
        
[36113]
36113


In [47]:
print(repo_set)

[36113]


In [48]:
#Take this out of quotes if you want to manually assign a repo_id number(s)
#repo_set = [25440]

## Query for Issue Analysis

In [ ]:
df_issues = pd.DataFrame()

for repo_id in repo_set: 

    pr_query = salc.sql.text(f"""
                SELECT
                    r.repo_name,
					i.issue_id AS issue, 
					i.gh_issue_number AS issue_number,
					i.gh_issue_id AS gh_issue,
					i.created_at AS created, 
					i.closed_at AS closed
                FROM
                	repo r,
                    issues i
                WHERE
                	r.repo_id = i.repo_id AND
                    i.repo_id = \'{repo_id}\'
        """)
    df_current_repo = pd.read_sql(pr_query, con=engine)
    df_issues = pd.concat([df_issues, df_current_repo])

df_issues = df_issues.reset_index()
df_issues.drop("index", axis=1, inplace=True)
        
df_issues.head()

,repo_name,issue,issue_number,gh_issue,created,closed
0,augur,91116,1525,1063293122,2021-11-25 08:33:28,NaT
1,augur,91121,1518,1055344486,2021-11-16 21:21:05,NaT
2,augur,91125,1514,1052511565,2021-11-13 01:32:55,NaT
3,augur,91128,1510,1041280595,2021-11-01 15:09:06,NaT
4,augur,91136,1495,1027800478,2021-10-15 20:55:07,NaT
